# Get Dark Area Values from Across SSA

In [1]:
import ee
import geemap

## Authenticate & Initialize GEE

Requires a [Google Cloud Project](https://console.cloud.google.com/projectcreate) and to enable the [Earth Engine API](https://console.cloud.google.com/apis/api/earthengine.googleapis.com) for the project. Find detailed instructions [here](https://book.geemap.org/chapters/01_introduction.html#earth-engine-authentication).

In [2]:
ee.Initialize()

## Create a Global Nightlights Map

In [3]:
m = geemap.Map(
    center=[0, 0], 
    zoom=3, 
    basemap = 'Esri.WorldImagery',
    height = 800
)

# add nightlights median
# https://developers.google.com/earth-engine/datasets/catalog/NOAA_VIIRS_DNB_MONTHLY_V1_VCMSLCFG
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2020-01-01', '2024-01-01'))
nighttime = dataset_night.select('avg_rad')
image_night = nighttime.median()
nighttimeVis = {'min': 0.0, 'max': 2.0}
m.addLayer(image_night, nighttimeVis, 'Nighttime', True)

m.add_basemap('CartoDB.VoyagerOnlyLabels')

m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

## Save Down Global Polygon

In [29]:
# create a ee feature rectangle from given coordinates
# globe_rect = ee.Geometry.Rectangle([-60, -40, 60, 40])

# # add to map
# m.addLayer(globe_rect, {}, 'Globe Rectangle')

In [39]:
# grab the last shape drawn
globe_poly_feat = m.draw_last_feature
# export the ee.Feature object to a geojson
geemap.ee_to_geojson(globe_poly_feat, 'data/dark_bg/globe_poly_feat.geo.json')

In [4]:

# read the geojson file
globe_poly_feat = geemap.geojson_to_ee('data/dark_bg/globe_poly_feat.geo.json')
# add to map
# m.add_layer(globe_poly_feat, {'color': 'blue', 'opacity': 0.3}, 'Globe Polygon')


## Sample

### Global Points

In [5]:
globe_pts = ee.FeatureCollection.randomPoints(globe_poly_feat, 5000, 44)
# export the ee.FeatureCollection object to a geojson
geemap.ee_to_geojson(globe_pts, 'data/dark_bg/globe_pts.geo.json')
# add to map
m.addLayer(globe_pts, {'color':'red'}, 'Globe Points')

### Ocean Points

In [96]:
# get ocean with SST dataset
sst = ee.ImageCollection('JAXA/GCOM-C/L3/OCEAN/SST/V3') \
                .filterDate('2023-06-01', '2024-01-01') \
                .filter(ee.Filter.eq('SATELLITE_DIRECTION', 'D')) \
                .mean() \
                .multiply(0.0012).add(-10)

vis = {
  'bands': ['SST_AVE'],
  'min': 0,
  'max': 30,
  'palette': ['000000', '005aff', '43c8c8', 'fff700', 'ff0000'],
}

m.addLayer(sst, vis, 'Sea Surface Temperature')


In [115]:

ocean_pts = sst.sample(
    region = globe_poly_feat,
    numPixels = 6133,
    scale = 10000,
    seed = 44,
    geometries = True,
    dropNulls = True
)

# print the length of the feature collection
print(ocean_pts.size().getInfo())

# export the ee.FeatureCollection object to a geojson
geemap.ee_to_geojson(ocean_pts, 'data/dark_bg/ocean_pts.geo.json')


4518


### Land Points

In [105]:
landcover = ee.ImageCollection('ESA/WorldCover/v200').first()
# m.addLayer(landcover, {}, 'Land Cover')

In [112]:
land_pts = landcover.sample(
    region = globe_poly_feat,
    numPixels = 16170,
    seed = 44,
    geometries = True,
    dropNulls = True
)

print(land_pts.size().getInfo())


4922


In [113]:
geemap.ee_to_geojson(land_pts, 'data/dark_bg/land_pts.geo.json')

### Visualize

In [6]:
globe_pts = geemap.geojson_to_ee('data/dark_bg/globe_pts.geo.json')
ocean_pts = geemap.geojson_to_ee('data/dark_bg/ocean_pts.geo.json')
land_pts = geemap.geojson_to_ee('data/dark_bg/land_pts.geo.json')

# check out on map
m.add_layer(globe_pts, {'color':'red'}, 'Globe Points', False)
m.add_layer(ocean_pts, {'color':'blue'}, 'Ocean Points')
m.add_layer(land_pts, {'color':'green'}, 'Land Points')


## Get the Nightlights Values

### Buffer

In [117]:
globe_pts = geemap.geojson_to_ee('data/dark_bg/globe_pts.geo.json')
ocean_pts = geemap.geojson_to_ee('data/dark_bg/ocean_pts.geo.json')
land_pts = geemap.geojson_to_ee('data/dark_bg/land_pts.geo.json')

# create a 1km buffer around each
globe_pts = globe_pts.map(lambda f: f.buffer(1000))
ocean_pts = ocean_pts.map(lambda f: f.buffer(1000))
land_pts = land_pts.map(lambda f: f.buffer(1000))

In [119]:
# get the image collection of nightlight images
dataset_night = ee.ImageCollection('NOAA/VIIRS/DNB/MONTHLY_V1/VCMSLCFG') \
                  .filter(ee.Filter.date('2013-01-01', '2024-06-01'))
# select the avg_rad band
nighttime_ic = dataset_night.select('avg_rad')
# stack the images into a single image
nighttime_ic_stack = nighttime_ic.toBands()
nighttime_ic_stack

In [120]:
# apply the reduceRegions for dark
globe_pts_lights = nighttime_ic_stack.reduceRegions(collection=globe_pts, reducer=ee.Reducer.mean())
ocean_pts_lights = nighttime_ic_stack.reduceRegions(collection=ocean_pts, reducer=ee.Reducer.mean())
land_pts_lights = nighttime_ic_stack.reduceRegions(collection=land_pts, reducer=ee.Reducer.mean())

In [121]:
# change each feature back to the original point geometry at the center of the buffer
globe_pts_lights = globe_pts_lights.map(lambda f: f.setGeometry(f.geometry().centroid()))
ocean_pts_lights = ocean_pts_lights.map(lambda f: f.setGeometry(f.geometry().centroid()))
land_pts_lights = land_pts_lights.map(lambda f: f.setGeometry(f.geometry().centroid()))


In [122]:

# export the ee.FeatureCollection object to a geojson
geemap.ee_to_geojson(globe_pts_lights, 'data/dark_bg/globe_pts_lights.geo.json')
geemap.ee_to_geojson(ocean_pts_lights, 'data/dark_bg/ocean_pts_lights.geo.json')
geemap.ee_to_geojson(land_pts_lights, 'data/dark_bg/land_pts_lights.geo.json')